<a href="https://colab.research.google.com/github/nliza/FP_DataMining/blob/main/scraping_femaleDaily.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import csv
import requests
from bs4 import BeautifulSoup

In [ ]:
def main():
    base_url = "https://reviews.femaledaily.com/products/moisturizer/gel/skintific/5x-ceramide-barrier-repair-moisture-gel-moisturizer"
    result = []

    # loop dari halaman 1 sampai 4
    for page in range(1, 120):
        url = f"{base_url}?cat=&cat_id=0&age_range=&skin_type=&skin_tone=&skin_undertone=&hair_texture=&hair_type=&order=newest&page={page}"
        response = requests.get(url)

        if response.status_code == 200:
            soup = BeautifulSoup(response.content, "html.parser")

            # semua elemen review
            reviews = soup.find_all("div", class_="text-wrapper")
            info_wrappers = soup.find_all("div", class_="information-wrapper")
            review_cards = soup.find_all("div", class_="review-card")

            for i, review in enumerate(reviews):
                try:
                    username = review.find("p", class_="profile-username").text.strip()
                    profile_age = review.find("p", class_="profile-age").text.strip()
                    profile_description = review.find("p", class_="profile-description").text.strip()
                except AttributeError:
                    username, profile_age, profile_description = "N/A", "N/A", "N/A"

                # usage period dan purchase point
                try:
                    usage_period_tag = info_wrappers[i].select_one("p:has(i.icon-ic_clock) b")
                    usage_period = usage_period_tag.text.strip() if usage_period_tag else "N/A"
                except (AttributeError, IndexError):
                    usage_period = "N/A"

                try:
                    purchase_point_tag = info_wrappers[i].select_one("p:has(i.icon-ic_shop) b")
                    purchase_point = purchase_point_tag.text.strip() if purchase_point_tag else "N/A"
                except (AttributeError, IndexError):
                    purchase_point = "N/A"

                #  rating dari jumlah bintang
                try:
                    stars = len(review_cards[i].select('.cardrv-starlist i.icon-ic_big_star_full'))
                except (AttributeError, IndexError):
                    stars = 0

                # status rekomendasi
                try:
                    recommend = review_cards[i].find('p', class_='recommend')
                    if recommend and recommend.find('i', class_='icon-ic_thumbs_up'):
                        recommendation_status = 'Ya'
                    elif recommend and recommend.find('i', class_='icon-ic_thumbs_down'):
                        recommendation_status = 'Tidak'
                    else:
                        recommendation_status = 'Tidak Ada Status'
                except (AttributeError, IndexError):
                    recommendation_status = 'N/A'

                # review
                try:
                    review_text = review_cards[i].find('p', class_='text-content')
                    review_text = review_text.get_text(strip=True) if review_text else 'No Review Text'
                except (AttributeError, IndexError):
                    review_text = 'N/A'

                # menyimpan hasil
                value = {
                    'Username': username,
                    'Profile Age': profile_age,
                    'Profile Description': profile_description,
                    'Usage Period': usage_period,
                    'Purchase Point': purchase_point,
                    'Stars': stars,
                    'Recommend': recommendation_status,
                    'Review': review_text
                }
                result.append(value)
        else:
            print(f"Failed to fetch page {page}: {response.status_code}")


    # menyimpan ke CSV
    if result:
        keys = result[0].keys()
        path = "/content/drive/MyDrive/Female Daily/review_femaleDaily.csv"
        with open(path, "w", newline="", encoding="utf-8") as output_file:
            dict_writer = csv.DictWriter(output_file, keys)
            dict_writer.writeheader()
            dict_writer.writerows(result)
        print(f"Reviews saved to '{path}'")
    else:
        print("No data to save.")

if __name__ == "__main__":
    main()


Reviews saved to '/content/drive/MyDrive/Female Daily/review_femaleDaily.csv'
